OOOOOOOH boy we made a GPT and had our fun, but now we need to dive into the world of tokenization, head first. Welcome to hell =)
Tokenization is at the heart of much weirdness of LLMs. Do not brush it off.

- Why can't LLM spell words? Tokenization.
- Why can't LLM do super simple string processing tasks like reversing a string? Tokenization.
- Why is LLM worse at non-English languages (e.g. Japanese)? Tokenization.
- Why is LLM bad at simple arithmetic? Tokenization.
- Why did GPT-2 have more than necessary trouble coding in Python? Tokenization.
- Why did my LLM abruptly halt when it sees the string "<|endoftext|>"? Tokenization.
- What is this weird warning I get about a "trailing whitespace"? Tokenization.
- Why the LLM break if I ask it about "SolidGoldMagikarp"? Tokenization.
- Why should I prefer to use YAML over JSON with LLMs? Tokenization.
- Why is LLM not actually end-to-end language modeling? Tokenization.
- What is the real root of suffering? Tokenization.

Example string:

In [ ]:
Tokenization is at the heart of much weirdness of LLMs. Do not brush it off.

127 + 677 = 804
1275 + 6773 = 8041

Egg.
I have an Egg.
egg.
EGG.

만나서 반가워요. 저는 OpenAI에서 개발한 대규모 언어 모델인 ChatGPT입니다. 궁금한 것이 있으시면 무엇이든 물어보세요.

for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)

In [1]:
"안녕하세요 👋 (hello in Korean!)"

# How do we feed this text into Python? A string is defined as immutable sequences of Unicode code points.
# Immutable means a state that cannot be modified after it's created. Making a string means there is no going back.
# Any operation in Python that modifies the string actually just makes a new one.
# Unicode codepoints are defined by the Unicode consortium according to the Unicode standard, which is ~150k characters,
# what the characters look like, and what integers represent those characters.
# We can get the integer representation of a character in Python by using ord()

'안녕하세요 👋 (hello in Korean!)'

In [2]:
[ord(x) for x in "안녕하세요 👋 (hello in Korean!)"]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

In [3]:
# Cool, so each charater has an integer defined to it. Why don't we just use those integers as our tokens
# and not even build a tokenizer? 
# 1. Our vocabulary would be super long (~150k)
# 2. The Unicode standard is changing over time

# To solve this, we will turn to encodings. The Unicode consortium defines 3 different encodings: UTF-8,
# UTF-16, and UTF-32. These take unicode text and translate into binary data, or byte streams. We will
# use the UTF-8 encoding (its backwards compatable to ASCI encoding and the 16/32 encodings are more wasteful).

list("안녕하세요 👋 (hello in Korean!)".encode("utf-8")) # gives the raw bytes of the string according to utf-8

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

In [ ]:
# It would be great to directly feed these byte streams into the transformer, but the transformer architecture
# doesn't allow for it. There is some work being done for a tokenization free transformer, but it is in
# the early stages.
# Instead, we are going to use the byte pair encoding algorithm =) It generally works as follows:
# Suppose you have the following bytes to be encoded (11 bytes with a vocab of 4)
# aaabdaaabac
# Identify the byte pairs that happen the most often and replace with a byte that is not used in the data (Z)
# Now our bytes turn into
# ZabdZabac (9 bytes with a vocab of 5)
# We can repeat this process, replacing 'ab' with Y
# ZYdZYac (7 bytes with vocab of 6)
# We can now replace "XY" with "X" to get
# XdXac (5 bytes with vocab of 7)
# This is the furthest this data can be compressed, since there are no more pairs that occur more than once.

In [6]:
# Lets get to the implementation

text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # converts to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print('---')
print(tokens)
print('length:', len(tokens))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140

In [7]:
from collections import Counter

def find_most_common_pair(tokens):
    pair_counts = Counter()
    # Iterate through all adjacent pairs
    for i in range(len(tokens) - 1):
        pair = (tokens[i], tokens[i + 1])
        pair_counts[pair] += 1
    
    # Find the most common pair
    if pair_counts:
        most_common_pair, _ = pair_counts.most_common(1)[0]
        return most_common_pair
    else:
        return None

print(find_most_common_pair(tokens))  # Example usage


(101, 32)
